In [31]:
import pandas as pd
import numpy as np
from sklearn import feature_selection, linear_model, metrics
from sklearn.model_selection import train_test_split


In [32]:
data = pd.read_excel('data_ford_price.xlsx')
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


# preprocessing


In [33]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace=True)

In [34]:
x = data.drop(columns='price')
y = data['price']


# RFE

In [35]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=40)
model = linear_model.LinearRegression()
rfe_selector = feature_selection.RFE(estimator=model, n_features_to_select=3, step=1)
rfe_selector.fit(X_train, y_train)
rfe_columns = list(rfe_selector.get_feature_names_out())
rfe_columns

['year', 'cylinders', 'lat']

In [36]:
model.fit(X_train[rfe_columns], y_train)
y_preds = model.predict(X_test[rfe_columns])
print(f"MAE для отбора признаков RFE: {metrics.mean_absolute_error(y_test, y_preds):.2f}")


MAE для отбора признаков RFE: 5096.57


### `RFE: 5096.57`

# SelectKBest

In [37]:
selector = feature_selection.SelectKBest(feature_selection.f_regression, k=3)
selector.fit(X_train, y_train)
sk_best_cols = list(selector.get_feature_names_out())
sk_best_cols

['year', 'cylinders', 'odometer']

In [30]:
model = linear_model.LinearRegression()
model.fit(X_train[sk_best_cols], y_train)
y_preds = model.predict(X_test[sk_best_cols])
print(f"MAE для отбора признаков SelectKBest: {metrics.mean_absolute_error(y_test, y_preds):.2f} ")

MAE для отбора признаков SelectKBest: 4708.95 


### `SKBEST: 4708.95`

## По метрике MAE метод отбора SelectKBEst c оценочной функцией f_reggression показал ощутимо более хороший результат, нежели метод RFE.